In [1]:
import sys
sys.path.append("src/")

import pandas as pd
import numpy as np
import tensorflow as tf

import os 

cwd = os.getcwd()
os.chdir("../src")
import tensorflow_utils as tf_utils
import utils as utils
from dataset_manager import DataSetManager

os.chdir(cwd)


/home/acevedo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [ ]:
a = tf.constant(2)
b = tf.constant(3)

c = sess.run(a+b)
print(c)
sess.close()

In [2]:

DIM = 512  # model dimensionality
GEN_DIM = 100  # output dimension of the generator
DIS_DIM = 1  # outptu dimension fo the discriminator
FIXED_GENERATOR = False  # wheter to hold the generator fixed at ral data plus Gaussian noise, as in the plots in the paper
LAMBDA = .1  # smaller lambda makes things faster for toy tasks, but isn't necessary if you increase CRITIC_ITERS enough
BATCH_SIZE = 256   # batch size
ITERS = 10000 #100000 # how many generator iterations to train for
FREQ = 250  # sample frequency




CRITIC_ITERS = 5  # homw many critic iteractions per generator iteration


def Generator_Softmax(n_samples,  name='gen'):

    with tf.variable_scope(name):
        noise = tf.random_normal([n_samples, GEN_DIM])
        output01 = tf_utils.linear(noise, DIM, name='fc-1')
        output01 = tf_utils.relu(output01, name='relu-1')
        
        output02 = tf_utils.linear(output01, DIM, name='fc-2')
        output02 = tf_utils.relu(output02, name='relu-2')
        
        output03 = tf_utils.linear(output02, DIM, name='fc-3')
        output03 = tf_utils.relu(output03, name='relu-3')
        
        output04 = tf_utils.linear(output03, GEN_DIM, name='fc-4')

        # Reminder: a logit can be modeled as a linear function of the predictors
        output05 = tf.nn.softmax(output04, name = 'softmax-1')

        
        return output05
        

def Discriminator(inputs, is_reuse=True, name='disc'):
    with tf.variable_scope(name, reuse=is_reuse):
        print('is_reuse: {}'.format(is_reuse))
        output01 = tf_utils.linear(inputs, DIM, name='fc-1')
        output01 = tf_utils.relu(output01, name='relu-1')

        output02 = tf_utils.linear(output01, DIM, name='fc-2')
        output02 = tf_utils.relu(output02, name='relu-2')

        output03 = tf_utils.linear(output02, DIM, name='fc-3')
        output03 = tf_utils.relu(output03, name='relu-3')

        output04 = tf_utils.linear(output03, DIS_DIM, name='fc-4')
        
        return output04
    
real_data = tf.placeholder(tf.float32, shape=[None, GEN_DIM])
fake_data = Generator_Softmax(BATCH_SIZE)

disc_real = Discriminator(real_data, is_reuse=False)
disc_fake = Discriminator(fake_data)

disc_cost = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)
gen_cost = - tf.reduce_mean(disc_fake)

# WGAN gradient penalty parameters

alpha = tf.random_uniform(shape=[BATCH_SIZE, 1], minval=0., maxval=1.)
interpolates = alpha*real_data + (1.-alpha) * fake_data
disc_interpolates = Discriminator(interpolates)
gradients = tf.gradients(disc_interpolates, [interpolates][0])
slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
gradient_penalty = tf.reduce_mean((slopes - 1)**2)

disc_cost += LAMBDA * gradient_penalty
    
disc_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='disc')
gen_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='gen')


gen/fc-1/add   [256, 512]
gen/fc-2/add   [256, 512]
gen/fc-3/add   [256, 512]
gen/fc-4/add   [256, 100]
is_reuse: False
disc/fc-1/add   [None, 512]
disc/fc-2/add   [None, 512]
disc/fc-3/add   [None, 512]
disc/fc-4/add   [None, 1]
is_reuse: True
disc_1/fc-1/add   [256, 512]
disc_1/fc-2/add   [256, 512]
disc_1/fc-3/add   [256, 512]
disc_1/fc-4/add   [256, 1]
is_reuse: True
disc_2/fc-1/add   [256, 512]
disc_2/fc-2/add   [256, 512]
disc_2/fc-3/add   [256, 512]
disc_2/fc-4/add   [256, 1]


# Traning Operations

In [3]:

# Use place holders for the learning rate
#WGAN Training operations

disc_lr = tf.placeholder(tf.float32, shape=()) # 1e-4
gen_lr = tf.placeholder(tf.float32, shape=()) # 1e-4

disc_train_op = tf.train.AdamOptimizer(learning_rate=disc_lr, beta1=0.5, beta2=0.9).minimize(disc_cost, var_list=disc_vars)

if len(gen_vars) > 0:
    gen_train_op = tf.train.AdamOptimizer(learning_rate=gen_lr, beta1=0.5, beta2=0.9).minimize(gen_cost, var_list=gen_vars)
else:
    gen_train_op = tf.no_op()


In [4]:
partiton_n = pd.read_csv('random_amounts.csv', header=None).values.flatten()
train_set =  pd.read_csv('../data/experiment_march_21/train_set.csv', header=None).values

np.random.seed(42)
indices = np.random.randint(0,train_set.shape[0], size=partiton_n[0]).tolist()
print(np.sum(indices))


my_ds = DataSetManager(train_set[indices])


556624454


# Triangualr CLR

In [ ]:
cycle = np.floor(1+iterations/(2*step_size))
x = np.abs(iterations/step_size - 2*cycle + 1)
lr = base_lr + (max_lr-base_lr)*np.maximum(0, (1-x))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
from clr import CyclicLR
import numpy as np

In [ ]:
pasos = 20000

magic_lr = CyclicLR(base_lr=10**-3.8, max_lr=10**-3.72, step_size=pasos)

In [ ]:
def train_step(clr_object):
    current_lr = clr_object.clr()
    # Sess run training operation
    clr_object.on_batch_end()
    return current_lr

In [ ]:
record_lr = np.array([ train_step(magic_lr) for k in range(2*pasos)])

In [ ]:
%matplotlib inline
plt.plot(record_lr)

# GAN Trainin Loop

In [6]:
desired_epochs = 1000

iters_per_epoch = my_ds.num_examples/BATCH_SIZE
total_iters = np.ceil(desired_epochs*iters_per_epoch)

print(total_iters)

critic_iters = np.round((5/6)*total_iters)
gen_iters = np.round((1/6)*total_iters)

63254.0


In [9]:
ITERS = total_iters
#magic_lr2 = CyclicLR(base_lr=0.01, max_lr=0.05, step_size=pasos)

ITERS = int(total_iters)
# Train loop
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #
    # DUCT TAPE SOLUTION
    # iter_ = 0

    """
    while my_ds.epochs_completed < desired_epochs:
        iter_ +=1    
    """
    gen_lr_ = CyclicLR(base_lr=10**-3.8, max_lr=10**-3.72, step_size=gen_iters)
    disc_lr_ = CyclicLR(base_lr=10**-3.8, max_lr=10**-3.72, step_size=critic_iters)
    for iter_ in range(ITERS):
        
        print(str(100*(iter_/ITERS)))
         
        batch_data, disc_cost_ = None, None

        previous_epoch =  my_ds.epochs_completed 

        # train critic
        for i_ in range(CRITIC_ITERS):
            batch_data =  my_ds.next_batch(BATCH_SIZE) # data_gen.__next__()
            disc_cost_, _ = sess.run([disc_cost, disc_train_op], feed_dict={real_data: batch_data, disc_lr:disc_lr_.clr() })
            disc_lr_.on_batch_end()

        # train generator
        sess.run(gen_train_op, feed_dict={gen_lr : gen_lr_.clr()})   
        gen_lr_.on_batch_end()

        gen_cost2 = sess.run(gen_cost)   

        current_epoch =  my_ds.epochs_completed 

        condition2 = current_epoch % 5 == 0

          


    if iter_ == ITERS:
        print("Current iter "+str(iter))
        #session_saver.save(sess, model_path)

0.0
0.0015809276883675342
0.0031618553767350684
0.0047427830651026025
0.006323710753470137
0.007904638441837671
0.009485566130205205
0.01106649381857274
0.012647421506940273
0.014228349195307806
0.015809276883675342
0.017390204572042874
0.01897113226041041
0.020552059948777943
0.02213298763714548
0.02371391532551301
0.025294843013880547
0.026875770702248076
0.028456698390615612
0.030037626078983144
0.031618553767350684
0.033199481455718216
0.03478040914408575
0.03636133683245328
0.03794226452082082
0.039523192209188346
0.041104119897555885
0.04268504758592342
0.04426597527429096
0.04584690296265848
0.04742783065102602
0.049008758339393554
0.050589686027761094
0.052170613716128626
0.05375154140449615
0.05533246909286369
0.056913396781231224
0.05849432446959876
0.06007525215796629
0.06165617984633383
0.06323710753470137
0.06481803522306889
0.06639896291143643
0.06797989059980396
0.0695608182881715
0.07114174597653904
0.07272267366490656
0.0743036013532741
0.07588452904164164
0.0774654567

KeyboardInterrupt: 

# LR Finder function

In [ ]:
def find_lr(session, cost_operation, optimize_operation, ds_man, net_type:str, init_value = 1e-8, final_value=10., beta = 0.98):
    num = len(trn_loader)-1
    mult = (final_value / init_value) ** (1/num)
    lr = init_value
    
    avg_loss = 0.
    best_loss = 0.
    batch_num = 0
    losses = []
    log_lrs = []
    for data in trn_loader:
        batch_num += 1
        #As before, get the loss for this mini-batch of inputs/outputs
        
        if net_type == 'GEN':
            loss = sess.run(cost_operation)
        else:
            batch_data = ds_man.next_batch(256)
            loss = sess.run(cost_operation, feed_dict={real_data: batch_data} )
                
        #Compute the smoothed loss
        avg_loss = beta * avg_loss + (1-beta) *loss.data[0]
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        #Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > 4 * best_loss:
            return log_lrs, losses
        #Record the best loss
        if smoothed_loss < best_loss or batch_num==1:
            best_loss = smoothed_loss
        #Store the values
        losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
        
        #Do one optimization step
        
        if net_type == 'GEN':
            sess.run(optimize_operation,  feed_dict={gen_lr: lr} )
        else:
            
            sess.run(optimize_operation,  feed_dict={disc_lr: lr} )
            
        #Update the lr for the next step
        lr *= mult
        
    return log_lrs, losses

In [ ]:
import math

def log_keeping(avg_loss,beta,loss,best_loss, batch_num,losses, log_lrs, lr ):
    #Compute the smoothed loss
    avg_loss = beta * avg_loss + (1-beta) *loss
    smoothed_loss = avg_loss / (1 - beta**batch_num)
    #Stop if the loss is exploding
    if batch_num > 1 and smoothed_loss > 4 * best_loss:
        return log_lrs, losses
    #Record the best loss
    if smoothed_loss < best_loss or batch_num==1:
        best_loss = smoothed_loss
    #Store the values
    losses.append(smoothed_loss)
    log_lrs.append(math.log10(lr))
    

def find_gan_lr( ds_man, init_value = 1e-8, final_value=10., beta = 0.98):
    #  cost_operation, optimize_operation


    num = np.round(ds_man.num_examples/256) # Iters for an epoch
    mult = (final_value / init_value) ** (1/num)
    lr_disc = init_value
    lr_gen = init_value

    batch_num = 0

    gen_avg_loss = 0.
    gen_best_loss = 0.
    gen_losses = []
    gen_log_lrs = []

    disc_avg_loss = 0.
    disc_best_loss = 0.
    disc_losses = []
    disc_log_lrs = []
        
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())

        for data in range(int(num)):
            if data%100==0:
                print("Iter "+str(data))
            batch_num += 1
            #As before, get the loss for this mini-batch of inputs/outputs
            

            gen_loss = sess.run(gen_cost)

            batch_data = ds_man.next_batch(256)
            disc_loss = sess.run(disc_cost, feed_dict={real_data: batch_data} )
            
            # DISC
            log_keeping(disc_avg_loss, beta, disc_loss, disc_best_loss, batch_num, disc_losses, disc_log_lrs, lr_disc )
            # GEN 
            log_keeping(gen_avg_loss,beta, gen_loss,gen_best_loss, batch_num, gen_losses, gen_log_lrs, lr_gen )
            
            #Do one optimization step

            # train critic
            for i_ in range(5):
                batch_data = ds_man.next_batch(256)
                disc_cost_ = sess.run( disc_train_op, feed_dict={real_data: batch_data,disc_lr: lr_disc} )

            sess.run(gen_train_op,  feed_dict={gen_lr: lr_gen} )
        
            #Update the lr for the next step
            lr_disc *= mult
            lr_gen *= mult
            
        return gen_log_lrs, gen_losses, disc_log_lrs, disc_losses

In [ ]:
gen_log_lrs, gen_losses, disc_log_lrs, disc_losses = find_gan_lr(my_ds)

In [ ]:
#%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
len(gen_log_lrs)


In [ ]:
%matplotlib widget
plt.plot(gen_log_lrs, np.multiply(-1,gen_losses))

In [ ]:

%matplotlib widget
plt.plot(disc_log_lrs[3:10], disc_losses[3:10])

In [ ]:
len(disc_log_lrs)